# NewsAPI  - Getting articles
This API can get news articles from different sources and different topics.  

Documentation for Python API can be found at:  
https://github.com/mattlisiv/newsapi-python



In [1]:
# Get your free API key from https://newsapi.org/, just need to sign up for an account
key = "56ba1070c68b4258a0bb25638215c516"

In [2]:
from newsapi import NewsApiClient
import json
from confluent_kafka import Producer

bootstrap_servers = "localhost:9092"

# Create a Kafka producer to send data
kafka_producer = Producer({'bootstrap.servers': bootstrap_servers})
def produce_message(topic, message):
    kafka_producer.produce(topic=topic, value=message.encode())

# Initialize API endpoint
newsapi = NewsApiClient(api_key="56ba1070c68b4258a0bb25638215c516")

# Define the list of media sources
sources = 'bbc-news,cnn,fox-news,nbc-news,the-guardian-uk,the-new-york-times,the-washington-post,usa-today,independent,daily-mail,hindustan-times'

# /v2/everything - note there are other parameters you can set
all_articles = newsapi.get_everything(q='Canada', #  Search query term (optional)
                                      sources=sources, # List of identifiers (IDs) for the
                                      language='en') # The 2-letter ISO code of the language

# Print the titles of the articles
for article in all_articles['articles']:
    produce_message('news-topic', article['title'])

kafka_producer.flush()

In [ ]:
from confluent_kafka import Consumer, KafkaError

def print_error(err):
    print("Error: {}".format(err))

def print_message(msg):
    print("topic = {}, partition = {}, offset = {}, key = {}, value = {}".format(
        msg.topic(), msg.partition(), msg.offset(),
        msg.key(), msg.value()))

def consume_messages(conf):
    consumer = Consumer(conf)
    try:
        consumer.subscribe([topic])
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print_error(msg.error())
            else:
                print_message(msg)
                consumer.commit(msg)
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        consumer.close()

topic = 'news-topic'
conf = {'bootstrap.servers': "localhost:9092", 'group.id': "News"}



In [2]:
pip install confluent-kafka

Note: you may need to restart the kernel to use updated packages.
